In [1]:
import Fast_similarity_matrix_func as F #put Fast_similarity_matrix_func.py in the same folder
import numpy as np
import concurrent.futures
import multiprocessing
import time
import plotly.express as px
from scipy.sparse import coo_matrix
from sys import getsizeof

multiprocessing.cpu_count()


24

In [5]:
#create synthetic dataset 
N = 20000 #number of data points
D=4 #dimensionality of data
tau = 0.05 #optional similarity cutoff 
dat=np.random.rand(N,D)

In [6]:
#baseline algorithm 0

#create synthetic dataset 
# N = 20000 #number of data points
# D=4 #dimensionality of data
# tau = 0.05 #optional similarity cutoff 
# dat=np.random.rand(N,D)

dat.dtype
# np.savetxt('dat.csv',dat, delimiter=',', fmt='%f')
 #similarity matrix to be filled in
    
start = time.perf_counter()
W = np.zeros((N,N), dtype=float) #similarity matrix to be filled in
#fill lower triangular part only
for i in np.arange(1,N):
    for j in np.arange(i):
        W[i,j] = F.dist(dat[i,:],dat[j,:])
        
sigm = lambda x : np.exp(-x**2/(2*0.3**2)) #gaussian similarity function or kernel
Q = sigm(W)
Q[Q==1]=0 #set upper triangle to 0


#threshold to increase sparsity, sacrificing some information
Qthresh = Q.copy()
Qthresh[Qthresh<=tau] = 0
#convert to COOrdinate format sparse matrix
Qthresh_coo = coo_matrix(Qthresh)
finish = time.perf_counter()
print(f'Finished in {round(finish-start, 5)} second(s)')
print(f'Size {getsizeof(Qthresh_coo)} bytes')


Finished in 0.1375 second(s)
Size 48 bytes


In [4]:
# fig = px.imshow(Qthresh_coo.toarray()[:20,:20])
# fig.show()

#store as sparse matrix
Q_coo = coo_matrix(Q)

#check memory use
print(getsizeof(Q))
print(getsizeof(Q_coo))
print(getsizeof(Qthresh))
print(getsizeof(Qthresh_coo))
print(Qthresh_coo.row)

# print(getsizeof(Q_coo.data) + getsizeof(Q_coo.row) + getsizeof(Q_coo.col))
# print(getsizeof(Qthresh_coo.data) + getsizeof(Qthresh_coo.row) + getsizeof(Qthresh_coo.col))

print(Q.nbytes)
print(Q_coo.data.nbytes + Q_coo.row.nbytes + Q_coo.col.nbytes)
print(Qthresh_coo.data.nbytes + Qthresh_coo.row.nbytes + Qthresh_coo.col.nbytes)# print(getsizeof(Qthresh_coo.data))
# print(getsizeof(Qthresh_coo.toarray()))
# print(getsizeof(Qthresh_coo.todense()))
# Qthresh_coo.todense().dtype
# Qthresh_coo.toarray().dtype
# Qthresh_coo.todense()
# Qthresh_coo.toarray()

3200000112
48
3200000112
48
[    1     3     4 ... 19999 19999 19999]
3200000000
3199840000
1373356896


In [4]:
# algorithm 1: delegate the interior loop of each row to different processors. return a coo sparse matrix result for each row of the similarity matrix which we will join together later

N = 20000 #number of data points
D=4 #dimensionality of data
tau = 0.05 #optional similarity cutoff 
# dat=np.random.rand(N,D)

start = time.perf_counter()

ROW = list(np.arange(1,N))
TAU = [tau for _ in range(N)]

data=[]
rows=[]
cols=[]

with concurrent.futures.ProcessPoolExecutor() as executor:
    results_generator = executor.map(F.row_similarity, [dat for _ in range(N)], ROW, TAU) #
    
#append each peice of data sent back from the workers to a list of arrays
    for i in results_generator:
        if i[0]: #if nonempty
            data.append(np.array(i[0]))
            rows.append(np.array(i[1]))
            cols.append(np.array(i[2]))
        
# print(data)
    data = np.concatenate(data, axis = 0)
    rows = np.concatenate(rows, axis = 0)
    cols = np.concatenate(cols, axis = 0)
    Q_PPEmap = coo_matrix((data, (rows, cols)), shape=(N,N)) #including the shape is important because othewise the stored matrix may be truncated (if there are rows/cols with all zeros!)

finish = time.perf_counter()
print(f'Finished in {round(finish-start, 5)} second(s)')
print(f'Size {getsizeof(Q_PPEmap)} bytes')


Finished in 228.75163 second(s)
Size 48 bytes


In [83]:
# Q_PPEmap.shape
# Q_PPEmap_size = Q_PPEmap.data.nbytes + Q_PPEmap.row.nbytes + Q_PPEmap.col.nbytes
# print(Q_PPEmap_size)
print(getsizeof(Q_PPEmap))
fig = px.imshow(Q_PPEmap.toarray()[:20,:20])
fig.show()

#check the matricies are equal
np.sum(np.sum(Q_PPEmap.todense()-Qthresh_coo.todense()))

48


In [7]:
#Algorithm 2: vectorize taking the difference between points by making each dimension have its own difference matrix. very memory intensive but may be fastttt

N = 20000 #number of data points
D=4 #dimensionality of data
tau = 0.05 #optional similarity cutoff 
# dat=np.random.rand(N,D)

start = time.perf_counter()

Nlist = [N for _ in range(D)]
Dlist = [d for d in range(D)]
M = np.zeros((N,N), dtype=float)
with concurrent.futures.ProcessPoolExecutor() as executor:
    result_generator = executor.map(F.componentwise_similarity, [dat for _ in range(D)], Dlist, Nlist)
    for d in result_generator:
        M += d
#         print(d)
    M = np.sqrt(M)
    sigm = lambda x : np.exp(-x**2/(2*0.3**2)) 
    M = sigm(M)
    M[M<tau]=0
    M[M==1]=0
    QM = coo_matrix(np.tril(M))

finish = time.perf_counter()
print(f'Finished in {round(finish-start, 5)} second(s)')
print(f'Size {getsizeof(QM)} bytes')


Finished in 1.58977 second(s)
Size 48 bytes


In [8]:
fig = px.imshow(QM.toarray()[:20,:20])
fig.show()
#check the matricies are equal
np.sum(np.sum(QM.todense()-Qthresh_coo.todense()))

0.0